# Dealing with Outliers in Numerical Columns – Using IQR Method

The Interquartile Range (IQR) method is a statistical technique used to identify outliers in a dataset. The IQR is the range between the first quartile (25th percentile) and the third quartile (75th percentile) of the data. It represents the range within which the middle 50% of the values lie.

**Key Steps in the IQR Method for Outlier Detection**

1. **Calculate the Q1 and Q3:** Q1 and Q3 (the 25th and 75th percentile, respectively) divide the ordered dataset into four equal parts. Q1 is the middle number between the smallest number and the median. Q3 is the middle value between the median and the highest value.

2. **Calculate the IQR:** The IQR is calculated as the difference between Q3 and Q1 (IQR = Q3 – Q1).

3. **Identify potential outliers:** Any data point that falls below Q1 – 1.5*IQR or above Q3 + 1.5*IQR is considered an outlier. The factor of 1.5 defines the "whiskers" of a box plot, which is used to visualize dispersion and skewness in the data. 

In the default setting, outliers are expected to fall above or below these whiskers. Adjusting the multiplier (1.5) allows for more or less flexibility in accounting for outliers, based on the specific data and use case.

> In other words, the IQR is a way of understanding the spread of the middle 50% of your data, and the method as a whole is a reliable way of detecting and handling outliers.

**Rough visualization:** <br>
We are interested in all values between the lower and upper whisker line, values outside would be outliers for us.

![Example Image](https://bitwise.exposed/wp-content/uploads/2024/04/image-1.png)

---

# Setup & References

## Import Libs

In [20]:
import pandas as pd
import numpy as np

## Use functions

| Function | Description |
| :--- | :--- |
| [quantile()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.quantile.html) | Return value at the given quantile over requested axis, a DataFrame level function. |
| [any()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.any.html) | Return whether any element is True over requested axis in a DataFrame. |

---
# Code Snippets

---

## Create Demo Data

#### Code

In [21]:
# Sample data with some outliers
data = {
    'Age': [25, 30, 35, 40, 45, 500],  # 500 is an outlier
    'Income': [50000, 70000, 80000, 90000, 100000, 9999999]  # 9999999 is an outlier
}
df = pd.DataFrame(data)

print("Original DataFrame:")
print(df)

Original DataFrame:
   Age   Income
0   25    50000
1   30    70000
2   35    80000
3   40    90000
4   45   100000
5  500  9999999


### Output review

We can obviously see the outliers in line 5, the whisker lines or the limit values were inserted beforehand, from which we consider a value to be an outlier. In the next code section we will calculate the whisker lines. <br><br>

|   | Age | Income   | Description                                 |
|---|-----|----------|---------------------------------------------|
|   | 12.5| 35,000 | **Lower Whisker limit in this sample data** |
| 0 | 25  | 50,000   | *Normal Data*                               |
| 1 | 30  | 70,000   | *Normal Data*                               |
| 2 | 35  | 80,000   | *Normal Data*                               |
| 3 | 40  | 90,000   | *Normal Data*                               |
| 4 | 45  | 100,000  | *Normal Data*                               |
|   | 62.5| 135,000  | **Upper Whisker limit in this sample data** |
| 5 | 500 | 9,999,999| *Outlier*                                   |



---
## Calculate Whisker

> *The following code cell is only to print out the whisker lines, resp. illustrate the limits for outliners* 

#### Cookbook Recipe

1. Define a function to calculate whisker limits in a dataset.
   - b. Calculate the first quartile (Q1) of the data.
   - c. Calculate the third quartile (Q3) of the data.
   - d. Compute the interquartile range (IQR) as Q3 - Q1.
   - e. Determine the lower whisker as Q1 minus 1.5 times the IQR.
   - f. Determine the upper whisker as Q3 plus 1.5 times the IQR.  

2. Apply the function to `Age` and `Income` columns of DataFrame to get the whisker values (lower and upper) for both columns.

3. Print the calculated lower and upper whisker limits for these columns.

#### Code

In [22]:
# 1. Define a function to calculate whisker limits in a dataframe.
def calculate_whiskers(df, multiplier=1.5):
    Q1 = df.quantile(0.25)  # 1.b
    Q3 = df.quantile(0.75)  # 1.c
    IQR = Q3 - Q1  # 1.d
    lower_whisker = Q1 - multiplier * IQR  # 1.e
    upper_whisker = Q3 + multiplier * IQR  # 1.f
    return lower_whisker, upper_whisker


# 2. Apply the function to 'Age' and 'Income' columns of dataframe.
age_lower, age_upper = calculate_whiskers(df['Age'])
income_lower, income_upper = calculate_whiskers(df['Income'])

# 3. Print the calculated lower and upper whisker limits for both columns.
print("Age - Lower Whisker: ", age_lower)
print("Age - Upper Whisker: ", age_upper)
print("Income - Lower Whisker: ", income_lower)
print("Income - Upper Whisker: ", income_upper)


Age - Lower Whisker:  12.5
Age - Upper Whisker:  62.5
Income - Lower Whisker:  35000.0
Income - Upper Whisker:  135000.0


---
## Clean outliners from data

#### Cookbook Recipe

1. Define a function to clean data from a dataset.
   - b. Calculate the first quartile (Q1) of the data.
   - c. Calculate the third quartile (Q3) of the data.
   - d. Compute the interquartile range (IQR) as Q3 - Q1.
   - e. Remove data points that are lower than Q1 minus 1.5 times the IQR.
   - f. Remove data points that are higher than Q3 plus 1.5 times the IQR.  

2. Apply the function to your dataframe to get a cleaned version of the data.

3. Print the cleaned dataframe.

#### Code

In [23]:
# 1.Define a function to clean data from a dataset.
def clean_data(df, multiplier=1.5):
    Q1 = df.quantile(0.25)  # 1.b
    Q3 = df.quantile(0.75)  # 1.c
    IQR = Q3 - Q1  # 1.d
    df = df[~((df < (Q1 - multiplier * IQR)) |
              (df > (Q3 + multiplier * IQR))).any(axis=1)]  # 1.e and 1.f

    return df


# 2. Apply the function to dataframe.
cleaned_df = clean_data(df)

# 3. Print the calculated lower and upper whisker limits for both columns.
print("\nCleaned DataFrame:")
print(cleaned_df)



Cleaned DataFrame:
   Age  Income
0   25   50000
1   30   70000
2   35   80000
3   40   90000
4   45  100000


---